In [1]:
import torch
import re

def load_ensemble(filepaths):
    def load_from_checkpoint(idx, filepath):
        parameters = torch.load(filepath)['state_dict']['bayesian_layer'].params
        parameters = {re.sub(r'model\.layers\.\d+\.', '', k): v for k, v in parameters.items() if v.numel() > 0}
        return parameters
    return [load_from_checkpoint(idx, filepath) for idx, filepath in enumerate(filepaths)]

parameters = load_ensemble(['logs/bayes/checkpoints/epoch=0-step=300.ckpt','logs/bayes/checkpoints/epoch=9-step=3000.ckpt', 'logs/bayes/checkpoints/epoch=3-step=1200.ckpt'])

In [2]:
%load_ext autoreload
%autoreload 2

from examples.bayes_llama3.llama3.modules.bayesllama import BayesLlamaForCausalLM
bayes_config = {'n_ensemble':3}

bayes = BayesLlamaForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", bayes_config=bayes_config).to("cuda:0")
bayes.load_bayesian_layers(parameters)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of BayesLlamaForCausalLM were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['model.bayesian_layers.0.input_layernorm.weight', 'model.bayesian_layers.0.mlp.down_proj.weight', 'model.bayesian_layers.0.mlp.gate_proj.weight', 'model.bayesian_layers.0.mlp.up_proj.weight', 'model.bayesian_layers.0.post_attention_layernorm.weight', 'model.bayesian_layers.0.self_attn.k_proj.weight', 'model.bayesian_layers.0.self_attn.o_proj.weight', 'model.bayesian_layers.0.self_attn.q_proj.weight', 'model.bayesian_layers.0.self_attn.v_proj.weight', 'model.bayesian_layers.1.input_layernorm.weight', 'model.bayesian_layers.1.mlp.down_proj.weight', 'model.bayesian_layers.1.mlp.gate_proj.weight', 'model.bayesian_layers.1.mlp.up_proj.weight', 'model.bayesian_layers.1.post_attention_layernorm.weight', 'model.bayesian_layers.1.self_attn.k_proj.weight', 'model.bayesian_layers.1.self_attn.o_proj.weight', 'model.bayesian_layers.1.self_attn.q_proj.weight'

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

inputs = "This is a test"
inputs = tokenizer(inputs, return_tensors="pt").input_ids

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
out = bayes(inputs.to("cuda"), return_dict=True, output_attentions=True, output_hidden_states=True, use_cache=True)

LlamaModel is using LlamaSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
32


In [5]:
out.logits[0].shape # (batch_size, sequence_length, vocab_size)
out.logits[1].shape # (n_ensemble, batch_size, sequence_length, vocab_size)

len(out.attentions) # 32 items
len(out.attentions[-1]) # Last item has attentions from n_ensemble layers

len(out.hidden_states) # 32 items 
out.hidden_states[-1].shape # (n_ensemble, batch_size, sequence_length, hidden_size)

len(out.past_key_values) # 32 items
out.past_key_values[-1][0].shape #Need to fix this 

torch.Size([1, 8, 15, 128])

In [9]:
# This is wrong
print(out.attentions[-1][0].shape) 
print(out.attentions[-1][-1].shape)

torch.Size([1, 32, 5, 5])
torch.Size([1, 32, 5, 15])
